# Regression
- TimeSeriesSplit 아닌 일반 교차검증 사용  
- train 2년, cv 3 / test 3개월

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 데이터셋 - DB 수주분석테이블에 맞춰서 변수명 변경하고 전처리

In [ ]:
!pip3 install pickle5
!pip install --upgrade pandas
!pip uninstall openpyxl
!pip install openpyxl

     |████████████████████████████████| 132 kB 2.7 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219275 sha256=fa0a18cad3db5ea8a480145b489d316767514528694243eb6d12ab8e566b64c1
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5
     |████████████████████████████████| 11.3 MB 3.5 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.4 which is incompatible.


Found existing installation: openpyxl 2.5.9
Uninstalling openpyxl-2.5.9:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/openpyxl-2.5.9.dist-info/*
    /usr/local/lib/python3.7/dist-packages/openpyxl/*
Proceed (y/n)? y
  Successfully uninstalled openpyxl-2.5.9
     |████████████████████████████████| 242 kB 3.1 MB/s 


In [ ]:
import pickle5 as pickle
with open('/content/drive/MyDrive/5조_스마트팩토리/mariadb_tables_pkl/수주분석테이블.pkl', "rb") as fh:
  수주분석테이블 = pickle.load(fh)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/5조_스마트팩토리/진유훈/증식데이터1806_2104.csv')
data.rename(columns={'일자':'납기일자','prod_name':'제품명','prediction':'예측중량','sold_quant':'판매수량'},inplace=True)
data.drop(data.columns[0],axis=1,inplace=True)
data['납기일자'] = pd.to_datetime(data['납기일자'])

# 제품별, 일별로 합계
data = pd.pivot_table(data,
                      index=['납기일자','제품명'],
                      values=['판매수량','예측중량'],
                      aggfunc='sum')


# sold_quant 0인 값 제거
data = data[data['판매수량']!=0]

# # prediction 마이너스 값 0으로 대체
minus_idx = data[data['예측중량']<0].index.tolist()
data.loc[minus_idx,'예측중량'] = 0

In [ ]:
data.reset_index(inplace=True)
data

In [ ]:
with open('/content/drive/MyDrive/5조_스마트팩토리/mariadb_tables_pkl/건축착공면적.pkl', "rb") as fh:
  건축착공면적 = pickle.load(fh)
건축착공면적['TIME'] = pd.to_datetime(건축착공면적['TIME'],format='%Y%m')
건축착공면적.set_index('TIME',inplace=True)
end_date = data['납기일자']
start_date = data['납기일자']-relativedelta(years=2)


In [ ]:
def year(x):
    date = str(x)
    year = date[:4]
    month = date[5:7]
    return int(year)

def month_(x):
    date = str(x)
    year = date[:4]
    month = date[5:7]
    return month

data['year'] = data['납기일자'].apply(year)
data['month'] = data['납기일자'].apply(month_)

건축착공면적.reset_index(inplace=True)
건축착공면적['month'] = 건축착공면적['TIME'].apply(month_)
건축착공면적.set_index('TIME',inplace=True)
건축착공면적
def need_stru(x):
    year = int(str(x)[:4])
    month = str(x)[5:7]
    need_df = 건축착공면적[f'{year-2}':f'{year-1}']
    month_need = need_df[need_df['month']==month]['건축착공면적'].mean()
    return month_need

data['착공지수'] = data['납기일자'].apply(need_stru)

result_data = data[['납기일자','제품명','예측중량','판매수량','착공지수']]
result_data

In [ ]:
############### 예측할때만 이거 쓰기 ###################
with open('/content/drive/MyDrive/5조_스마트팩토리/mariadb_tables_pkl/건축착공면적.pkl', "rb") as fh:
  건축착공면적 = pickle.load(fh)
건축착공면적['TIME'] = pd.to_datetime(건축착공면적['TIME'],format='%Y%m')
# today = datetime.today().strftime('%Y-%m-%d')
std_date = datetime.strptime('2021-04-25', '%Y-%m-%d')
start_date = std_date - relativedelta(years=2)
start_date = datetime.strftime(start_date,'%Y-%m-%d')
건축착공면적 = 건축착공면적[건축착공면적['TIME']>=start_date]
건축착공면적['month'] = 건축착공면적.TIME.dt.month
건축착공면적=pd.DataFrame(건축착공면적.groupby('month')['건축착공면적'].mean())
건축착공면적.reset_index(inplace=True)
건축착공면적


# 분석용 데이터셋 생성

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pickle5 as pickle

data = pd.read_csv('/content/drive/MyDrive/5조_스마트팩토리/진유훈/증식데이터1806_2104.csv')
data.rename(columns={'일자':'납기일자','prod_name':'제품명','prediction':'예측중량','sold_quant':'판매수량'},inplace=True)
data.drop(data.columns[0],axis=1,inplace=True)
data['납기일자'] = pd.to_datetime(data['납기일자'])

# 제품별, 일별로 합계
data = pd.pivot_table(data,
                      index=['납기일자','제품명'],
                      values=['판매수량','예측중량'],
                      aggfunc='sum')
data.reset_index(inplace=True)

# # sold_quant 0인 값 제거
# data = data[data['판매수량']!=0]

# # # prediction 마이너스 값 0으로 대체
# minus_idx = data[data['예측중량']<0].index.tolist()
# data.loc[minus_idx,'예측중량'] = 0

In [ ]:
data.head()

,납기일자,제품명,예측중량,판매수량
0,2018-06-01,CSA5000,49382.8000,43490.0
1,2018-06-01,PEMA-HR1000,131839.6600,60320.0
2,2018-06-01,PEMA-HR1500,58646.2420,7820.0
3,2018-06-01,PEMA-PCM2000,4376.7344,5530.0
4,2018-06-01,PEMA-PR1000,44807.1880,39170.0


In [ ]:
with open('/content/drive/MyDrive/5조_스마트팩토리/mariadb_tables_pkl/건축착공면적.pkl', "rb") as fh:
  건축착공면적 = pickle.load(fh)
건축착공면적['TIME'] = pd.to_datetime(건축착공면적['TIME'],format='%Y%m')
건축착공면적.set_index('TIME',inplace=True)
건축착공면적.head()

,건축착공면적
TIME,
2016-04-01,14455700.0
2016-05-01,13280000.0
2016-06-01,12792800.0
2016-07-01,12917200.0
2016-08-01,11521900.0


In [ ]:
def year(x):
    date = str(x)
    year = date[:4]
    month = date[5:7]
    return int(year)

def month_(x):
    date = str(x)
    year = date[:4]
    month = date[5:7]
    return month

data['year'] = data['납기일자'].apply(year)
data['month'] = data['납기일자'].apply(month_)

건축착공면적.reset_index(inplace=True)
건축착공면적['month'] = 건축착공면적['TIME'].apply(month_)
건축착공면적.set_index('TIME',inplace=True)
건축착공면적
def need_stru(x):
    year = int(str(x)[:4])
    month = str(x)[5:7]
    need_df = 건축착공면적[f'{year-2}':f'{year-1}']
    month_need = need_df[need_df['month']==month]['건축착공면적'].mean()
    return month_need

data['착공지수'] = data['납기일자'].apply(need_stru)
data.head()
# result_data = data[['납기일자','제품명','예측중량','판매수량','착공지수']]
# result_data

,납기일자,제품명,예측중량,판매수량,year,month,착공지수
0,2018-06-01,CSA5000,49382.8000,43490.0,2018,06,12401600.0
1,2018-06-01,PEMA-HR1000,131839.6600,60320.0,2018,06,12401600.0
2,2018-06-01,PEMA-HR1500,58646.2420,7820.0,2018,06,12401600.0
3,2018-06-01,PEMA-PCM2000,4376.7344,5530.0,2018,06,12401600.0
4,2018-06-01,PEMA-PR1000,44807.1880,39170.0,2018,06,12401600.0


In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pickle5 as pickle
from sklearn.preprocessing import MinMaxScaler
data = pd.read_csv('/content/drive/MyDrive/5조_스마트팩토리/진유훈/증식데이터1806_2104.csv')
data.rename(columns={'일자':'납기일자','prod_name':'제품명','prediction':'예측중량','sold_quant':'판매수량'},inplace=True)
data.drop(data.columns[0],axis=1,inplace=True)
data['납기일자'] = pd.to_datetime(data['납기일자'])

# 제품별, 일별로 합계
data = pd.pivot_table(data,
                      index=['납기일자','제품명'],
                      values=['판매수량','예측중량'],
                      aggfunc='sum')


# sold_quant 0인 값 제거
data = data[data['판매수량']!=0]

# # prediction 마이너스 값 0으로 대체
minus_idx = data[data['예측중량']<0].index.tolist()
data.loc[minus_idx,'예측중량'] = 0
data.reset_index(inplace=True)

# 건축착공면적 불러와서 전처리

with open('/content/drive/MyDrive/5조_스마트팩토리/mariadb_tables_pkl/건축착공면적.pkl', "rb") as fh:
  건축착공면적 = pickle.load(fh)
건축착공면적['TIME'] = pd.to_datetime(건축착공면적['TIME'],format='%Y%m')
건축착공면적.set_index('TIME',inplace=True)



def year(x):
    date = str(x)
    year = date[:4]
    month = date[5:7]
    return int(year)

def month_(x):
    date = str(x)
    year = date[:4]
    month = date[5:7]
    return month

data['year'] = data['납기일자'].apply(year)
data['month'] = data['납기일자'].apply(month_)

건축착공면적.reset_index(inplace=True)
건축착공면적['month'] = 건축착공면적['TIME'].apply(month_)
건축착공면적.set_index('TIME',inplace=True)
건축착공면적
def need_stru(x):
    year = int(str(x)[:4])
    month = str(x)[5:7]
    need_df = 건축착공면적[f'{year-2}':f'{year-1}']
    month_need = need_df[need_df['month']==month]['건축착공면적'].mean()
    return month_need

data['착공지수'] = data['납기일자'].apply(need_stru)

data = data[['납기일자','제품명','예측중량','판매수량','착공지수']]
data.set_index('납기일자',inplace=True)



# # sold_quant, prediction 로그변환
scaler = MinMaxScaler()
data[['착공지수']] = scaler.fit_transform(data[['착공지수']])
data[['판매수량','예측중량']] = np.log1p(data[['판매수량','예측중량']])

# # month, year drop
# data.drop(['month','year'],axis=1,inplace=True)

# # prod_name 원핫인코딩
data = pd.get_dummies(data)

In [ ]:
data.head()

,예측중량,판매수량,착공지수,제품명_AE,제품명_CSA4000,제품명_CSA5000,제품명_CSA5000F,제품명_PEMA-500FR,제품명_PEMA-580FX,제품명_PEMA-CR1000,제품명_PEMA-CSA5000,제품명_PEMA-HR1000,제품명_PEMA-HR1000F,제품명_PEMA-HR1000OPC,제품명_PEMA-HR1000S,제품명_PEMA-HR1500,제품명_PEMA-HR1500F,제품명_PEMA-PCM2000,제품명_PEMA-PCM2000B,제품명_PEMA-PCR3000E,제품명_PEMA-PCR3000N,제품명_PEMA-PR1000,제품명_PEMA-PR10001,제품명_PEMA-PR2000,제품명_PEMA-SN400,제품명_PEMA-SP1000,제품명_PEMA-SPR,제품명_PEMA-SR2000,제품명_PEMA-SR2000A,제품명_PEMA-SR3000F,제품명_PEMA-SR5000F,제품명_PR1000,제품명_SRE-110,제품명_SRE-200
납기일자,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-06-01,10.807378,10.680309,0.734254,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2018-06-01,11.789349,11.007436,0.734254,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2018-06-01,10.979296,8.964568,0.734254,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2018-06-01,8.384287,8.618124,0.734254,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2018-06-01,10.710146,10.575692,0.734254,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


# 추가 독립변수
- 월평균 기온(상관계수 낮아서 삭제)
- 월평균 강수량(상관계수 낮아서 삭제)

In [ ]:
# # 월평균 기온, 월평균 강수량 컬럼 추가

# def weather(data):
#   data = data.reset_index()
#   weather = pd.read_pickle('/content/drive/MyDrive/5조_스마트팩토리/진유훈/temp18_21.pkl')
#   weather.drop(['시도','신적설량'],axis=1,inplace=True)
#   weather['일자'] = pd.to_datetime(weather['일자'])

#   grouped_ta = weather.groupby(weather.일자.dt.month)['기온'].mean()
#   grouped_rn = weather.groupby(weather.일자.dt.month)['강수량'].mean()

#   # 월평균 기온
#   def m_temp(month):
#       if month in [1]: return grouped_ta.iloc[0]
#       elif month in [2]: return grouped_ta.iloc[1]   
#       elif month in [3]: return grouped_ta.iloc[2]
#       elif month in [4]: return grouped_ta.iloc[3]
#       elif month in [5]: return grouped_ta.iloc[4]
#       elif month in [6]: return grouped_ta.iloc[5]
#       elif month in [7]: return grouped_ta.iloc[6]
#       elif month in [8]: return grouped_ta.iloc[7]
#       elif month in [9]: return grouped_ta.iloc[8]
#       elif month in [10]: return grouped_ta.iloc[9]
#       elif month in [11]: return grouped_ta.iloc[10]
#       elif month in [12]: return grouped_ta.iloc[11]

#   # 월평균 강수량
#   def m_rain(month):
#       if month in [1]: return grouped_rn.iloc[0]
#       elif month in [2]: return grouped_rn.iloc[1]   
#       elif month in [3]: return grouped_rn.iloc[2]
#       elif month in [4]: return grouped_rn.iloc[3]
#       elif month in [5]: return grouped_rn.iloc[4]
#       elif month in [6]: return grouped_rn.iloc[5]
#       elif month in [7]: return grouped_rn.iloc[6]
#       elif month in [8]: return grouped_rn.iloc[7]
#       elif month in [9]: return grouped_rn.iloc[8]
#       elif month in [10]: return grouped_rn.iloc[9]
#       elif month in [11]: return grouped_rn.iloc[10]
#       elif month in [12]: return grouped_rn.iloc[11]

#   data['m_temp'] = data['납기일자'].dt.month.apply(m_temp)
#   data['m_rain'] = data['납기일자'].dt.month.apply(m_rain)
#   data = data.set_index('납기일자')
#   return data

# data = weather(data)

# 최적 모델 및 하이퍼 파라미터 탐색

In [ ]:
def Regression(data):
  from datetime import datetime
  from dateutil.relativedelta import relativedelta
  from sklearn.metrics import mean_squared_error , r2_score
  from sklearn.linear_model import Ridge, Lasso
  from sklearn.tree import DecisionTreeRegressor
  from sklearn.ensemble import RandomForestRegressor
  from xgboost import XGBRegressor
  from lightgbm import LGBMRegressor
  from sklearn.model_selection import GridSearchCV
  import joblib

  # 종속변수, 독립변수 분할
  y = data['판매수량']
  X = data.drop('판매수량',axis=1)

  # train, validation, test 분할
  test_end = data.index.max()
  test_start = test_end - relativedelta(months=3)
  train_end = test_start - relativedelta(days=1)
  train_start = test_end - relativedelta(years=2)
  
  X_test = X[test_start:test_end]
  y_test = y[test_start:test_end]
  X_train = X[train_start:train_end]
  y_train = y[train_start:train_end]

# 객체 생성
  ridge = Ridge(random_state=1004)
  lasso = Lasso(random_state=1004)
  dt_reg = DecisionTreeRegressor(random_state=1004)
  rf_reg = RandomForestRegressor(random_state=1004)
  xgb_reg = XGBRegressor(objective='reg:squarederror',random_state=1004)
  lgbm_reg = LGBMRegressor(random_state=1004)

  # 파라미터 설정
  ridge_parameters = {'alpha':[0.1,0.5,1.0]} # 0.5
  lasso_parameters = {'alpha':[0.0001,0.0003,0.0005]}
  dt_parameters = {'max_depth':[3,5,7]}
  rf_parameters = {'n_estimators':[300, 400, 500, 600, 700, 800, 900, 1000],
                   'min_samples_split':[25,50,75,100]}
  xgb_parameters = {'n_estimators':[100, 200,300,400],
                    'learning_rate':[0.01,0.05,0.1],
                    'max_depth':[2,4,6,8]}
  lgbm_parameters = {'n_estimators':[100,300,500,700,800,1000],
                    'learning_rate':[0.01, 0.05,0.1,0.5],
                    'colsample_bytree':[0.5,0.75,1.0]}

  reg_param = [(ridge,ridge_parameters),
               (lasso,lasso_parameters),
               (dt_reg,dt_parameters),
               (rf_reg,rf_parameters),
               (xgb_reg,xgb_parameters),
               (lgbm_reg,lgbm_parameters)]

  for reg, parameter in reg_param:
    grid_reg = GridSearchCV(reg, param_grid=parameter,
                            scoring='neg_mean_squared_error',
                            cv=3)
    
    grid_reg.fit(X_train, y_train)

    # 교차검증 결과 출력
    class_name = reg.__class__.__name__
    print(f'<<<<< {class_name} >>>>>')
    scores_df = pd.DataFrame(grid_reg.cv_results_)
    display(scores_df[['params','rank_test_score','mean_test_score']])
    print(f'{class_name} 최적 하이퍼 파라미터:',grid_reg.best_params_)
    print('{0} 최고 MSLE:{1:.4f}'.format(class_name,-1 * grid_reg.best_score_))
    print('{0} 최고 RMSLE:{1:.4f}'.format(class_name,np.sqrt(-1 * grid_reg.best_score_)))

    
    # test data에 최적 하이퍼 파라미터 적용하여 분석한 결과
    best_reg = grid_reg.best_estimator_
    pred = best_reg.predict(X_test)

    y_pred_expm = np.expm1(pred)
    y_test_expm = np.expm1(y_test)

    msle = mean_squared_error(y_test, pred)
    rmsle = np.sqrt(msle)
    accuracy = r2_score(y_test_expm, y_pred_expm)

    print('MLSE scores:{0:.4f}'.format(msle))
    print('RMLSE scores:{0:.4f}'.format(rmsle))
    print('테스트 데이터 세트 정확도:{:.4f}'.format(accuracy))
    print('='*80)
    print()

In [ ]:
Regression(data)

<<<<< Ridge >>>>>


,params,rank_test_score,mean_test_score
0,{'alpha': 0.1},3,-0.147873
1,{'alpha': 0.5},2,-0.147782
2,{'alpha': 1.0},1,-0.147689


Ridge 최적 하이퍼 파라미터: {'alpha': 1.0}
Ridge 최고 MSLE:0.1477
Ridge 최고 RMSLE:0.3843
MLSE scores:0.1388
RMLSE scores:0.3726
테스트 데이터 세트 정확도:0.8910

<<<<< Lasso >>>>>


,params,rank_test_score,mean_test_score
0,{'alpha': 0.0001},1,-0.147794
1,{'alpha': 0.0003},2,-0.147915
2,{'alpha': 0.0005},3,-0.148277


Lasso 최적 하이퍼 파라미터: {'alpha': 0.0001}
Lasso 최고 MSLE:0.1478
Lasso 최고 RMSLE:0.3844
MLSE scores:0.1387
RMLSE scores:0.3725
테스트 데이터 세트 정확도:0.8915

<<<<< DecisionTreeRegressor >>>>>


...

RandomForestRegressor 최적 하이퍼 파라미터: {'min_samples_split': 50, 'n_estimators': 1000}
RandomForestRegressor 최고 MSLE:0.0805
RandomForestRegressor 최고 RMSLE:0.2837
MLSE scores:0.0517
RMLSE scores:0.2274
테스트 데이터 세트 정확도:0.9543

<<<<< XGBRegressor >>>>>


,params,rank_test_score,mean_test_score
0,"{'learning_rate': 0.01, 'max_depth': 2, 'n_est...",48,-10.981195
1,"{'learning_rate': 0.01, 'max_depth': 2, 'n_est...",44,-1.557295
2,"{'learning_rate': 0.01, 'max_depth': 2, 'n_est...",37,-0.284867
3,"{'learning_rate': 0.01, 'max_depth': 2, 'n_est...",33,-0.113424
4,"{'learning_rate': 0.01, 'max_depth': 4, 'n_est...",45,-10.965842
5,"{'learning_rate': 0.01, 'max_depth': 4, 'n_est...",41,-1.556187
6,"{'learning_rate': 0.01, 'max_depth': 4, 'n_est...",38,-0.290226
7,"{'learning_rate': 0.01, 'max_depth': 4, 'n_est...",34,-0.113614
8,"{'learning_rate': 0.01, 'max_depth': 6, 'n_est...",46,-10.965972
9,"{'learning_rate': 0.01, 'max_depth': 6, 'n_est...",43,-1.556505


XGBRegressor 최적 하이퍼 파라미터: {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}
XGBRegressor 최고 MSLE:0.0832
XGBRegressor 최고 RMSLE:0.2884
MLSE scores:0.0533
RMLSE scores:0.2309
테스트 데이터 세트 정확도:0.9436

<<<<< LGBMRegressor >>>>>


,params,rank_test_score,mean_test_score
0,"{'colsample_bytree': 0.5, 'learning_rate': 0.0...",72,-0.357645
1,"{'colsample_bytree': 0.5, 'learning_rate': 0.0...",65,-0.124647
2,"{'colsample_bytree': 0.5, 'learning_rate': 0.0...",44,-0.099495
3,"{'colsample_bytree': 0.5, 'learning_rate': 0.0...",27,-0.094618
4,"{'colsample_bytree': 0.5, 'learning_rate': 0.0...",22,-0.093467
5,"{'colsample_bytree': 0.5, 'learning_rate': 0.0...",18,-0.092387
6,"{'colsample_bytree': 0.5, 'learning_rate': 0.0...",36,-0.096632
7,"{'colsample_bytree': 0.5, 'learning_rate': 0.0...",17,-0.092142
8,"{'colsample_bytree': 0.5, 'learning_rate': 0.0...",23,-0.093467
9,"{'colsample_bytree': 0.5, 'learning_rate': 0.0...",26,-0.094273


LGBMRegressor 최적 하이퍼 파라미터: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'n_estimators': 500}
LGBMRegressor 최고 MSLE:0.0834
LGBMRegressor 최고 RMSLE:0.2888
MLSE scores:0.0484
RMLSE scores:0.2200
테스트 데이터 세트 정확도:0.9519



# 최적 모델 저장

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sklearn.metrics import mean_squared_error , r2_score
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
import joblib

# 종속변수, 독립변수 분할
y = data['판매수량']
X = data.drop('판매수량',axis=1)

# train, validation, test 분할
test_end = data.index.max()
test_start = test_end - relativedelta(months=3)
train_end = test_start - relativedelta(days=1)
train_start = test_end - relativedelta(years=2)

X_test = X[test_start:test_end]
y_test = y[test_start:test_end]
X_train = X[train_start:train_end]
y_train = y[train_start:train_end]


# RandomForestRegressor & LGBM 혼합모델
rf_reg = RandomForestRegressor(random_state=1004)
rf_parameters = {'n_estimators':[50,100,300,500,1000],
                  'min_samples_split':[25,50,75,100]}
grid_rf = GridSearchCV(rf_reg, param_grid=rf_parameters,
                       scoring='neg_mean_squared_error',cv=3)
grid_rf.fit(X_train, y_train)
pred_rf = grid_rf.predict(X_test)


lgbm_reg = LGBMRegressor(random_state=1004)
lgbm_parameters = {'n_estimators':[100,300,500,700,800,1000],
                  'learning_rate':[0.01, 0.05,0.1,0.5],
                  'colsample_bytree':[0.5,0.75,1.0]}
grid_lgbm = GridSearchCV(lgbm_reg, param_grid=lgbm_parameters,
                       scoring='neg_mean_squared_error',cv=3)
grid_lgbm.fit(X_train, y_train)
pred_lgbm = grid_lgbm.predict(X_test)                  

# xgb_reg = XGBRegressor(objective='reg:squarederror',random_state=1004)
# xgb_parameters = {'n_estimators':[200,300,400],
#                     'learning_rate':[0.01,0.05,0.1],
#                     'max_depth':[2,4,6,8]}
# grid_xgb = GridSearchCV(xgb_reg, param_grid=xgb_parameters,
#                        scoring='neg_mean_squared_error',cv=3)
# grid_xgb.fit(X_train, y_train)
# pred_xgb = grid_xgb.predict(X_test)


pred = 0.5 * pred_rf + 0.5 * pred_lgbm

pred_expm = np.expm1(pred)
y_test_expm = np.expm1(y_test)

msle = mean_squared_error(y_test, pred)
rmsle = np.sqrt(msle)
accuracy = r2_score(y_test_expm, pred_expm)

print('RF & XGB 혼합모델 MLSE scores:{0:.4f}'.format(msle))
print('RF & XGB 혼합모델 RMLSE scores:{0:.4f}'.format(rmsle))
print('테스트 데이터 세트 정확도:{:.4f}'.format(accuracy))
print('='*80)
print()

RF & XGB 혼합모델 MLSE scores:0.0490
RF & XGB 혼합모델 RMLSE scores:0.2214
테스트 데이터 세트 정확도:0.9538



# 모델 저장

In [ ]:
from sklearn.externals import joblib
joblib.dump(grid_rf, "/content/drive/MyDrive/5조_스마트팩토리/권준기/배송이_예측모델/회귀모델_save_final/grid_rf.joblib.dat")
# joblib.dump(grid_lgbm, "/content/drive/MyDrive/5조_스마트팩토리/권준기/배송이_예측모델/회귀모델_save_final/grid_lgbm.joblib.dat")

['/content/drive/MyDrive/5조_스마트팩토리/권준기/배송이_예측모델/회귀모델_save_final/grid_rf.joblib.dat']

# 모델 저장(방법2)
  
코드 참고 : https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/  
** joblib 사용 시 DB에서 에러남/해결됨

In [ ]:
# import pickle

# filename1 = '/content/drive/MyDrive/5조_스마트팩토리/조현정/Regression_models/rf_reg.sav'
# pickle.dump(grid_rf, open(filename1, 'wb'))

# filename2 = '/content/drive/MyDrive/5조_스마트팩토리/조현정/Regression_models/lgbm_reg.sav'
# pickle.dump(grid_lgbm, open(filename2, 'wb'))


# # 모델 사용 예시
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, y_test)
# print(result)

## DB에서 중장기수주예측 불러와서 단기생판량 예측할 때

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import pickle5 as pickle
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta





with open('/content/drive/MyDrive/5조_스마트팩토리/mariadb_tables_pkl/중장기수주예측.pkl', "rb") as fh:
  중장기수주예측 = pickle.load(fh)

std_date = std_date = datetime.strptime("2021-04-25", "%Y-%m-%d")
end_date = std_date + relativedelta(days=95)
중장기수주예측 = 중장기수주예측[(중장기수주예측['납기일자']>=std_date) & (중장기수주예측['납기일자']<=end_date)]

df = 중장기수주예측.copy()
df.set_index('납기일자',inplace=True)
df['예측중량'] = np.log1p(df['예측중량'])
df.fillna(0,inplace=True)

with open('/content/drive/MyDrive/5조_스마트팩토리/mariadb_tables_pkl/건축착공면적.pkl', "rb") as fh:
  건축착공면적 = pickle.load(fh)
건축착공면적['TIME'] = pd.to_datetime(건축착공면적['TIME'],format='%Y%m')
# today = datetime.today().strftime('%Y-%m-%d')
std_date = datetime.strptime('2021-04-25', '%Y-%m-%d')
start_date = std_date - relativedelta(years=2)

def cat(df):
  df = df[df['TIME']>=start_date]
  df['month'] = df.TIME.dt.month
  df = pd.DataFrame(df.groupby('month')['건축착공면적'].mean())
  df.reset_index(inplace=True)
  return df

건축착공면적 = cat(건축착공면적)

df.reset_index(inplace=True)
df['month'] = df['납기일자'].dt.month
# data['year'] = data['납기일자'].dt.year
df = pd.merge(df, 건축착공면적, on='month', how='left')
df.set_index('납기일자',inplace=True)


df = pd.get_dummies(df)

df.reset_index(inplace=True)

X_test = df.drop(['납기일자','month'], axis =1)



#############################
grid_rf = joblib.load('/content/drive/MyDrive/5조_스마트팩토리/권준기/배송이_예측모델/회귀모델_save_final/grid_rf.joblib.dat')
#############################


# #############################
# grid_xgb = joblib.load('/content/drive/MyDrive/5조_스마트팩토리/권준기/배송이_예측모델/회귀모델_save/grid_xgb.joblib.dat')
#############################


pred_rf = grid_rf.predict(X_test)
# pred_xgb = grid_xgb.predict(X_test)
# pred_혼합 = 0.5 * pred_rf + 0.5 * pred_xgb


pred = pd.DataFrame(pred_rf,columns=['생산량'])
pred['생산량'] = np.ceil(np.expm1(pred.생산량))
중장기수주예측 = 중장기수주예측[['납기일자','제품명']]
중장기수주예측.reset_index(drop=True,inplace=True)
생판계획 = pd.concat([중장기수주예측,pred],axis=1)

pd.set_option('display.max_rows',None)
생판계획.sort_values(by='납기일자',ascending=True,inplace=True)
생판계획.reset_index(drop=True,inplace=True)
생판계획


,납기일자,제품명,생산량
0,2021-04-25,AE,1099.0
1,2021-04-25,SRE-200,12749.0
2,2021-04-25,SRE-110,2747.0
3,2021-04-25,PR1000,12367.0
4,2021-04-25,PEMA-SR5000F,4521.0
5,2021-04-25,PEMA-SR2000A,5823.0
6,2021-04-25,PEMA-SR2000,15522.0
7,2021-04-25,PEMA-SPR,2549.0
8,2021-04-25,PEMA-SP1000,4150.0
9,2021-04-25,PEMA-SN400,7227.0


In [ ]:
X_test

,예측중량,month,건축착공면적,제품명_AE,제품명_CSA4000,제품명_CSA5000,제품명_CSA5000F,제품명_PEMA-500FR,제품명_PEMA-580FX,제품명_PEMA-CR1000,제품명_PEMA-CSA5000,제품명_PEMA-HR1000,제품명_PEMA-HR1000F,제품명_PEMA-HR1000OPC,제품명_PEMA-HR1000S,제품명_PEMA-HR1500,제품명_PEMA-HR1500F,제품명_PEMA-PCM2000,제품명_PEMA-PCM2000B,제품명_PEMA-PCR3000E,제품명_PEMA-PCR3000N,제품명_PEMA-PR1000,제품명_PEMA-PR10001,제품명_PEMA-PR2000,제품명_PEMA-SN400,제품명_PEMA-SP1000,제품명_PEMA-SPR,제품명_PEMA-SR2000,제품명_PEMA-SR2000A,제품명_PEMA-SR3000F,제품명_PEMA-SR5000F,제품명_PR1000,제품명_SRE-110,제품명_SRE-200
0,7.045811,4,11331100.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,8.036001,4,11331100.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7.672395,4,11331100.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9.012270,4,11331100.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,9.020443,4,11331100.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1680,8.865530,7,9839680.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1681,9.415743,7,9839680.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1682,9.702472,7,9839680.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1683,8.510495,7,9839680.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
